In [15]:
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

In [16]:
model_name_list = ['Prior', 'LR', 'SVC','RF']#, 'INN']

In [17]:
round_precision = 4
# dtype_dict = {'binary_crossentropy': float, 'brier_score': float, 'accuracy': float, 'TN': int, 'TP': int, 'FN': int, 'FP': int}

In [18]:
with open('../data/data_test.pt', 'rb') as file:
    X_test, y_test = pickle.load(file)

print(f'{X_test.shape = }')
print(f'{y_test.shape = }')

X_test.shape = (622230, 33)
y_test.shape = (622230, 2)


In [19]:
y_pred = {}
for model_name in model_name_list:
    with open(f'./predictions/{model_name}.pt', 'rb') as file:
        y_pred[model_name] = pickle.load(file)

In [20]:
results = {}

for i, y_label in enumerate(['hosp', 'death']):
    results[y_label] = {}
    
    for model_name in model_name_list:
        results[y_label][model_name] = {}

        # average performance over 5 runs
        if model_name in ['INN']:
            log_loss_list = [log_loss(y_test[:, i], y_pred[model_name][y_label][j][:, 1]) for j in range(5)]
            results[y_label][model_name]['binary_crossentropy'] = f'{np.mean(log_loss_list).round(round_precision)} ± {np.std(log_loss_list).round(round_precision)}'
            results[y_label][model_name]['binary_crossentropy_sortable'] = np.mean(log_loss_list).round(round_precision)

            brier_score_list = [brier_score_loss(y_test[:, i], y_pred[model_name][y_label][j][:, 1]) for j in range(5)]
            results[y_label][model_name]['brier_score'] = f'{np.mean(brier_score_list).round(round_precision)} ± {np.std(brier_score_list).round(round_precision)}'

            accuracy_list = [accuracy_score(y_test[:, i], y_pred[model_name][y_label][j][:, 1].round()) for j in range(5)]
            results[y_label][model_name]['accuracy'] = f'{np.mean(accuracy_list).round(round_precision)} ± {np.std(accuracy_list).round(round_precision)}'

            CMs = [confusion_matrix(y_test[:, i], (y_pred[model_name][y_label][j][:, 1] > 0.5).astype(int)) for j in range(5)]
            results[y_label][model_name]['TN'] = f'{np.mean([CMs[j][0, 0] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][0, 0] for j in range(5)]).round(round_precision)}'
            results[y_label][model_name]['TP'] = f'{np.mean([CMs[j][1, 1] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][1, 1] for j in range(5)]).round(round_precision)}'
            results[y_label][model_name]['FP'] = f'{np.mean([CMs[j][0, 1] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][0, 1] for j in range(5)]).round(round_precision)}'
            results[y_label][model_name]['FN'] = f'{np.mean([CMs[j][1, 0] for j in range(5)]).round(round_precision)} ± {np.std([CMs[j][1, 0] for j in range(5)]).round(round_precision)}'
        # training is deterministic, no need to average
        else:
            results[y_label][model_name]['binary_crossentropy'] = log_loss(y_test[:, i], y_pred[model_name][y_label][:, 1]).round(round_precision)
            results[y_label][model_name]['binary_crossentropy_sortable'] = log_loss(y_test[:, i], y_pred[model_name][y_label][:, 1]).round(round_precision)
            results[y_label][model_name]['brier_score'] = brier_score_loss(y_test[:, i], y_pred[model_name][y_label][:, 1]).round(round_precision)
            results[y_label][model_name]['accuracy'] = accuracy_score(y_test[:, i], y_pred[model_name][y_label][:, 1].round()).round(round_precision)

            CM = confusion_matrix(y_test[:, i], (y_pred[model_name][y_label][:, 1] > 0.5).astype(int))
            results[y_label][model_name]['TN'] = CM[0, 0]
            results[y_label][model_name]['TP'] = CM[1, 1]
            results[y_label][model_name]['FP'] = CM[0, 1]
            results[y_label][model_name]['FN'] = CM[1, 0]

In [23]:
for y_label in ['hosp', 'death']:
    results[y_label]['Prior']['CPU'] = '-'
    results[y_label]['SVC']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'
    results[y_label]['RF']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'
    results[y_label]['LR']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'
    # results[y_label]['INN']['CPU'] = 'i7-6700k(4C/8T)@4.5GHz'

    results[y_label]['Prior']['GPU'] = '-'
    results[y_label]['SVC']['GPU'] = '-'
    results[y_label]['RF']['GPU'] = '-'
    results[y_label]['LR']['GPU'] = '-'
    # results[y_label]['INN']['GPU'] = 'GTX980Ti@1353/7010MHz'

    results[y_label]['Prior']['Time (HH:mm)'] = '-'
    results[y_label]['SVC']['Time (HH:mm)'] = '00:45'
    results[y_label]['RF']['Time (HH:mm)'] = '02:00'
    results[y_label]['LR']['Time (HH:mm)'] = '03:00'
    # results[y_label]['INN']['Time (HH:mm)'] = '26:00'

# Results

## Hospitalization

In [24]:
pd.DataFrame(results['hosp']).T.sort_values('binary_crossentropy_sortable')

,binary_crossentropy,binary_crossentropy_sortable,brier_score,accuracy,TN,TP,FP,FN,CPU,GPU,Time (HH:mm)
SVC,0.2083,0.2083,0.0559,0.9339,579242,1887,405,40696,i7-6700k(4C/8T)@4.5GHz,-,00:45
Prior,0.2496,0.2496,0.0638,0.9316,579647,0,0,42583,-,-,-
RF,0.3564,0.3564,0.053,0.9369,576393,6577,3254,36006,i7-6700k(4C/8T)@4.5GHz,-,02:00
LR,0.3994,0.3994,0.1071,0.9257,568582,7438,11065,35145,i7-6700k(4C/8T)@4.5GHz,-,03:00


## Fatality

In [25]:
pd.DataFrame(results['death']).T.sort_values('binary_crossentropy_sortable')

,binary_crossentropy,binary_crossentropy_sortable,brier_score,accuracy,TN,TP,FP,FN,CPU,GPU,Time (HH:mm)
RF,0.0348,0.0348,0.0086,0.9896,613692,2090,286,6162,i7-6700k(4C/8T)@4.5GHz,-,02:00
SVC,0.0411,0.0411,0.0104,0.9885,613726,1322,252,6930,i7-6700k(4C/8T)@4.5GHz,-,00:45
LR,0.0518,0.0518,0.0105,0.9886,613963,1145,15,7107,i7-6700k(4C/8T)@4.5GHz,-,03:00
Prior,0.0705,0.0705,0.0131,0.9867,613978,0,0,8252,-,-,-
